In [56]:
## Install NeMo
BRANCH = 'v1.8.2'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install jiwer

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml --no-check-certificate

  Cloning https://github.com/NVIDIA/NeMo.git (to revision v1.8.2) to /tmp/pip-install-zipwbu83/nemo-toolkit_9a4b78a326fd4ad7858e35ccfe6aef3d
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-zipwbu83/nemo-toolkit_9a4b78a326fd4ad7858e35ccfe6aef3d
^C
ERROR: Operation cancelled by user
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
mkdir: cannot create directory ‘configs’: File exists
--2022-05-18 17:22:03--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.8.2/examples/asr/conf/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4163 (4.1K) [text/plain]
Saving to: ‘configs/config.yaml.3’

conf

In [7]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
from omegaconf import DictConfig
import pytorch_lightning as pl
import time 
import torchaudio

In [65]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2022-05-18 17:34:18 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2022-05-18 17:34:18 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2022-05-18 17:34:18 common:747] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-18 17:34:19 features:259] PADDING: 16
[NeMo I 2022-05-18 17:34:19 features:276] STFT using torch
[NeMo I 2022-05-18 17:34:20 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [66]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data/audio_files_train/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data/train_script.txt'
train_manifest_path = '/home/dgxuser/Desktop/kailin/location_data/train_manifest.json'

audio_paths = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)
    audio_paths.append(audio_dir + str(n).zfill(4) + '_resampled.wav')
    n = n+1

with open(train_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [67]:
# test before training 
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths: 
        line = script.readline().strip()
        files = [path]
        print(f'{line} was predicted as {quartznet.transcribe(paths2audio_files=files)[0]}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Bishan Bishan was predicted as be shun be shun


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Boon Lay Boon Lay was predicted as bonlay bun lay


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Tiong Bahru Tiong Bahru was predicted as tong baru kong baru


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Tuas Tuas was predicted as twis twisce


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Yishun Yishun was predicted as he should he should


In [68]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [69]:
# def enable_bn_se(m):
#     if type(m) == nn.BatchNorm1d:
#         m.train()
#         for param in m.parameters():
#             param.requires_grad_(True)

#     if 'SqueezeExcite' in type(m).__name__:
#         m.train()
#         for param in m.parameters():
#             param.requires_grad_(True)

# quartznet.encoder.freeze()
# quartznet.encoder.apply(enable_bn_se)
# print("Model encoder has been frozen, and batch normalization has been unfrozen")

# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = train_manifest_path

# lr decrease for fine-tuning 
new_opt = copy.deepcopy(params['model']['optim'])
new_opt['lr'] = 0.001

# Use the smaller learning rate we set before
quartznet.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=50, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo W 2022-05-18 17:34:22 modelPT:475] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-05-18 17:34:22 modelPT:587] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2022-05-18 17:34:22 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2022-05-18 17:34:22 audio_to_text_dataset:41] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-05-18 17:34:22 audio_to_text_dataset:41] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-05-18 17:34:22 collections:186] Dataset loaded with 9 files totalling 0.01 hours
[NeMo I 2022-05-18 17:34:22 collections:187] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-18 17:34:22 audio_to_text_dataset:41] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-05-18 17:34:22 audio_to_text_dataset:41] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-05-18 17:34:22 collections:186] Dataset loaded with 9 files totalling 0.01 hours
[NeMo I 2022-05-18 17:34:22 collections:187] 0 files were filtered totalling

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-05-18 17:34:22 modelPT:475] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [70]:
# test after training 
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths: 
        line = script.readline().strip()
        files = [path]
        print(f'{line} was predicted as {quartznet.transcribe(paths2audio_files=files)[0]}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Bishan Bishan was predicted as bishan bishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Boon Lay Boon Lay was predicted as boon lay boon lay


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Tiong Bahru Tiong Bahru was predicted as tiong bahru tiong bahru


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Tuas Tuas was predicted as tuas tuas


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Yishun Yishun was predicted as yishun yishun


In [71]:
# test WER of Singapore English Corpus 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
speakers = ['0010', '0016', '0019', '0039', '0042']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [72]:
error = wer(transcript, predicted)
print('----- Training without Freezing Encoder -----')
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

----- Training without Freezing Encoder -----
Word Error Rate: 0.7291953050650312
Number of Testcases (Lines): 2373


In [73]:
# Freeze encoder and try again 
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
    
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen, and batch normalization has been unfrozen")

# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = train_manifest_path

# lr decrease for fine-tuning 
new_opt = copy.deepcopy(params['model']['optim'])
new_opt['lr'] = 0.001

# Use the smaller learning rate we set before
quartznet.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=50, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-05-18 17:43:04 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2022-05-18 17:43:04 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2022-05-18 17:43:04 common:747] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-18 17:43:04 features:259] PADDING: 16
[NeMo I 2022-05-18 17:43:04 features:276] STFT using torch
[NeMo I 2022-05-18 17:43:05 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


[NeMo W 2022-05-18 17:43:05 modelPT:475] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


Model encoder has been frozen, and batch normalization has been unfrozen
[NeMo I 2022-05-18 17:43:05 modelPT:587] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2022-05-18 17:43:05 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2022-05-18 17:43:05 audio_to_text_dataset:41] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-05-18 17:43:05 audio_to_text_dataset:41] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-05-18 17:43:05 collections:186] Dataset loaded with 9 files totalling 0.01 hours
[NeMo I 2022-05-18 17:43:05 collections:187] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-18 17:43:05 audio_to_text_dataset:41] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-05-18 17:43:05 audio_to_text_dataset:41] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-05-18 17:43:05 collections:186] Dataset loaded with 9 files totalling 0.01 hours
[NeMo I 2022-05-18 17:43:05 collections:187] 0 files were filtered totalling

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [75]:
# test after training 
script_dir = '/home/dgxuser/Desktop/kailin/location_data/train_script.txt'
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths: 
        line = script.readline().strip()
        files = [path]
        print(f'{line} was predicted as {quartznet.transcribe(paths2audio_files=files)[0]}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Bishan Bishan was predicted as bisha bishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Boon Lay Boon Lay was predicted as bunlay bun lay


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Tiong Bahru Tiong Bahru was predicted as tiin byru tainbyru


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Tuas Tuas was predicted as twa twa


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Yishun Yishun was predicted as ishon ye shun


In [76]:
# test WER of Singapore English Corpus 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
speakers = ['0010', '0016', '0019', '0039', '0042']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:
error = wer(transcript, predicted)
print('----- Training with Frozen Encoder -----')
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

----- Training with Frozen Encoder -----
Word Error Rate: 0.5197560889640831
Number of Testcases (Lines): 2373


In [ ]:
#less iterations 
#change optimisation parameters (learning rate)
#freeze more layers 
#dataset too small -> overfitting 